This notebook parses the data exported from your Strava profile and plots it on an interactive window via the Google Maps API. 

First, we we need to install the dependancies for this notebook. First of all, install [GPSbabel](https://www.gpsbabel.org/download.html), which we use to parse and convert `.gz` and `.tcx` files to the same `.gpx` format. On Linux, this can done via 
```bash
apt install gpsbabel
```
Most Python dependances can be installed by running `pip install -r requirements.txt` (also included in the first Python cell). However, we'll be using a custom version of `gpxpy`, which parses `.gpx` files. In the main package, 'trckpts' points without longitude and latitude fields raise an error for the entire file. The [forked version](https://github.com/watermarkhu/gpxpy) does not raise this error, and deals with such points later. 
Clone the fork and install with pip in developer mode. 
```bash
git clone https://github.com/watermarkhu/gpxpy
pip install -e ./gpxpy
```


In [ ]:
!pip install -r requirements.txt
import gpxpy
from progiter import ProgIter as prog
from gpxconverter import Main
from plotter import PlotApp
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

Set folder of unzipped export folder and convert all to `gpx` files. 

In [ ]:
folder = "/mnt/c/Users/water/Downloads/export_25412727/"
files = Main(folder, folder + "activities", folder + "activities.csv")

Parse `gpx` files to a Pandas dataframe. We use GeoPy to find the location of each track. We limit the number of calls to the location finder to 1 call per second. Change the value for `user_agent` if neccessary. 

In [ ]:
data = []
locator = RateLimiter(Nominatim(user_agent="strava_mapper2").reverse, min_delay_seconds=1)
# locator = Nominatim(user_agent="strava").reverse

for filename, activity_type in prog(files.items()):
    with open(folder + filename, "r") as file:
        gpxdata = gpxpy.parse(file)

    mydict = None
    for track in gpxdata.tracks:
        for segment in track.segments:
            for point in segment.points:
                if point.latitude and point.longitude:
                    if not mydict:
                        locdict = locator("{}, {}".format(point.latitude, point.longitude), language='en').raw
                        mydict = dict(
                            year = point.time.year,
                            month = point.time.month,
                            weekday = point.time.weekday(),
                            hour = point.time.hour
                        )
                        for key in ["country", "state", "city"]:
                            mydict[key] = locdict["address"][key] if key in locdict["address"] else "Unknown"
                    data.append(dict(
                        latitude = point.latitude,
                        longitude = point.longitude,
                        time = point.time,
                        type = activity_type,
                        file = filename,
                        **mydict
                    ))

df = pd.DataFrame(data)
print("Dataframe ready")

Plot dynamically on Google Maps via Gmaps. A Google Maps API key is required. The documentation is provided by [Gmaps](https://jupyter-gmaps.readthedocs.io/en/latest/). For the categories, one can choose any combination from year, month, weekday, hour, time, type and file. Add categories in previous cell if needed. 

In [ ]:

with open("api_key", "r") as file:
    api_key = file.read()

categories = ["city", "type"]

map = PlotApp(api_key, df, categories)
map.render()

The heatmap gradient can be changed by supplying a list of colors. 

In [ ]:
map.heatmap.gradient = [
    (0,0,0,0),
    'blue',
    'purple',
    'red'
]

If the one wants to customize the style of the background map, we will need to install the version of Gmaps of this [pull request](https://github.com/pbugnion/gmaps/pull/330). The pull request is fully functional, but development on Gmaps is seemingly inactive. 

First uninstall installed version of gmaps. 
```bash
pip uninstall gmaps
```
To install this version, we'll need clone the repo and build from source (npm is required). 
```bash
git clone https://github.com/krystofcelba/gmaps/
cd gmaps
git switch feature/add-support-for-styles
bash dev-install
```
The map style loaded via a json file. A custom style can be created [here](https://mapstyle.withgoogle.com/). Two styles are included in the styles folder. 

In [ ]:
with open("./styles/dark.json", "r") as file:
  style = file.read()
map.fig.styles = style